In [2]:

# Install necessary libraries
!pip install pyspark
!pip install python-dotenv

In [3]:

# Import necessary libraries
from google.colab import userdata
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os
from pyspark.sql.functions import explode, col, to_timestamp, substring
from pyspark.sql.types import StructType, StructField, StringType, LongType, BooleanType, ArrayType

In [4]:

# Load environment variables for Azure access information
load_dotenv("/content/azure_connection.env")

storage_account_name = os.getenv("AZURE_ACCOUNT_NAME")
storage_account_key = os.getenv("AZURE_STORAGE_KEY")
storage_container_name = "kaggle-datasets"
parquet_blob_name = "github-dataset-full.parquet"

In [5]:

# Creating Spark session
spark = SparkSession.builder \
    .appName("Read Parquet from Azure Blob Storage") \
    .config(f"spark.hadoop.fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key) \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.2,com.microsoft.azure:azure-storage:8.6.6") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

# Remove garbage error texts
spark.sparkContext.setLogLevel("ERROR")

In [6]:
# Set authentification for Spark to connect to Azure
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net",
    storage_account_key
)


In [7]:
# Read data to ensure data was properly saved
repo_list_df = spark.read.parquet(
    "wasbs://kaggle-datasets@matthewleffler1.blob.core.windows.net/clean_data/repo_list_data"
)

In [8]:

# Ensure data was saved
repo_list_df.show(10, truncate=False)

+-------+----------+-------------------+-------------------+-----------------------------------------------------------------------------------------+---------+----------------+---------------------------------------+-------------+--------+-------------+---------------------------------------+----------------+-------------+-------------------+---------+---------------------+-------------------+
|user_id|user_login|repo_created_at    |repo_default_branch|repo_description                                                                         |repo_fork|repo_forks_count|repo_full_name                         |repo_has_wiki|repo_id |repo_language|repo_license                           |repo_open_issues|repo_owner_id|repo_pushed_at     |repo_size|repo_stargazers_count|repo_updated_at    |
+-------+----------+-------------------+-------------------+-----------------------------------------------------------------------------------------+---------+----------------+---------------------------

In [9]:

# Read data to ensure data was properly saved
following_list_df = spark.read.parquet(
    "wasbs://kaggle-datasets@matthewleffler1.blob.core.windows.net/clean_data/following_list_data"
)


In [10]:

# Ensure data was saved
following_list_df.show(10, truncate=False)

+--------+---------------+------------+
|user_id |user_login     |following_id|
+--------+---------------+------------+
|3103473 |ashubhadani    |401908      |
|16760183|jandersonaraujo|16691304    |
|167845  |joelennon      |46539       |
|167845  |joelennon      |643307      |
|167845  |joelennon      |1073533     |
|167845  |joelennon      |2650468     |
|167845  |joelennon      |2650485     |
|167845  |joelennon      |2656153     |
|167845  |joelennon      |2656266     |
|167845  |joelennon      |2789596     |
+--------+---------------+------------+
only showing top 10 rows



In [11]:

# Read data to ensure data was properly saved
follower_list_df = spark.read.parquet(
    "wasbs://kaggle-datasets@matthewleffler1.blob.core.windows.net/clean_data/follower_list_data"
)

In [12]:
# Ensure data was saved
follower_list_df.show(10, truncate=False)

+--------+---------------+--------------+
|user_id |user_login     |follower_login|
+--------+---------------+--------------+
|15133929|blackstonep    |15222051      |
|15133929|blackstonep    |17273091      |
|15133929|blackstonep    |9455290       |
|15133929|blackstonep    |6808931       |
|6819477 |smartraysam    |25648077      |
|11713529|avielmenter    |15976384      |
|11713529|avielmenter    |28541828      |
|16760183|jandersonaraujo|1280437       |
|16760183|jandersonaraujo|26728605      |
|167845  |joelennon      |643307        |
+--------+---------------+--------------+
only showing top 10 rows



In [13]:

# Read data to ensure data was properly saved
commit_list_df = spark.read.parquet(
    "wasbs://kaggle-datasets@matthewleffler1.blob.core.windows.net/clean_data/commit_list_data"
)

In [14]:

# Ensure data was saved
repo_list_df.show(10, truncate=False)

+-------+----------+-------------------+-------------------+-----------------------------------------------------------------------------------------+---------+----------------+---------------------------------------+-------------+--------+-------------+---------------------------------------+----------------+-------------+-------------------+---------+---------------------+-------------------+
|user_id|user_login|repo_created_at    |repo_default_branch|repo_description                                                                         |repo_fork|repo_forks_count|repo_full_name                         |repo_has_wiki|repo_id |repo_language|repo_license                           |repo_open_issues|repo_owner_id|repo_pushed_at     |repo_size|repo_stargazers_count|repo_updated_at    |
+-------+----------+-------------------+-------------------+-----------------------------------------------------------------------------------------+---------+----------------+---------------------------

In [15]:

# Read data to ensure data was properly saved
non_list_df = spark.read.parquet(
    "wasbs://kaggle-datasets@matthewleffler1.blob.core.windows.net/clean_data/non_list_data"
)

In [16]:

# Ensure data was saved
repo_list_df.show(10, truncate=False)

+-------+----------+-------------------+-------------------+-----------------------------------------------------------------------------------------+---------+----------------+---------------------------------------+-------------+--------+-------------+---------------------------------------+----------------+-------------+-------------------+---------+---------------------+-------------------+
|user_id|user_login|repo_created_at    |repo_default_branch|repo_description                                                                         |repo_fork|repo_forks_count|repo_full_name                         |repo_has_wiki|repo_id |repo_language|repo_license                           |repo_open_issues|repo_owner_id|repo_pushed_at     |repo_size|repo_stargazers_count|repo_updated_at    |
+-------+----------+-------------------+-------------------+-----------------------------------------------------------------------------------------+---------+----------------+---------------------------

In [17]:
!pip install snowflake-connector-python pandas


✅ Query 1: Most Active Repositories by Commit Volume

Goal: Identify which repositories have the most commits — great for finding high-activity projects.

🔧 SQL Query (based on commit_list_df):

In [30]:
commit_list_df.createOrReplaceTempView("commits")
active_repos_df = spark.sql("""
    SELECT repo_name, COUNT(*) AS total_commits
    FROM commits
    GROUP BY repo_name
    ORDER BY total_commits DESC
    LIMIT 500
""")


In [31]:
active_repos_pd = active_repos_df.toPandas()

import snowflake.connector
conn = snowflake.connector.connect(
    user='bigdata228',
    password=userdata.get('SNOWFLAKE_PASSWORD'),
    account='BFDEIDG-MVB00946',
    warehouse='COMPUTE_WH',
    database='BIGDATA_GITHUB',
    schema='ANALYTICS',
    role='ACCOUNTADMIN'
)
cursor = conn.cursor()

cursor.execute("CREATE OR REPLACE TABLE most_active_repos (repo_name STRING, total_commits INT)")
cursor.executemany(
    "INSERT INTO most_active_repos (repo_name, total_commits) VALUES (%s, %s)",
    active_repos_pd.values.tolist()
)

cursor.close()
conn.close()
print("✅ Data saved to Snowflake.")


✅ Data saved to Snowflake.


 ✅ Query 2: Average Followers per Developer

 Goal: Measure developer reach/influence based on how many followers they have.

 🔧 SQL Query (based on follower_list_df):


In [32]:
follower_list_df.createOrReplaceTempView("followers")

influential_devs_df = spark.sql("""
    SELECT
        user_login AS developer,
        COUNT(follower_login) AS follower_count
    FROM followers
    GROUP BY user_login
    ORDER BY follower_count DESC
    LIMIT 500
""")


In [33]:
# Assume this DataFrame already exists from your Spark SQL
influential_devs_pd = influential_devs_df.toPandas()

# Reconnect to Snowflake (if needed)
import snowflake.connector
conn = snowflake.connector.connect(
    user='bigdata228',
    password=userdata.get('SNOWFLAKE_PASSWORD'),
    account='BFDEIDG-MVB00946',
    warehouse='COMPUTE_WH',
    database='BIGDATA_GITHUB',
    schema='ANALYTICS',
    role='ACCOUNTADMIN'
)
cursor = conn.cursor()

# Create + Insert: top_influential_developers
cursor.execute("CREATE OR REPLACE TABLE top_influential_developers (developer STRING, follower_count INT)")
cursor.executemany(
    "INSERT INTO top_influential_developers (developer, follower_count) VALUES (%s, %s)",
    influential_devs_pd.values.tolist()
)

cursor.close()
conn.close()
print("✅ Developer influence data saved to Snowflake.")


✅ Developer influence data saved to Snowflake.
